# Setup
Instalando e importando principais bibliotecas

In [94]:
# pip install basedosdados

In [95]:
import basedosdados as bd
import pandas as pd

In [96]:
billing_project_id = "curso-bigquery-361218"

# Resolvendo as questões

### 1. Quantos chamados foram abertos no dia 01/04/2023?

In [97]:
query = "SELECT * FROM `datario.adm_central_atendimento_1746.chamado` WHERE DATE(data_inicio) = '2023-04-01'"
df = bd.read_sql(query, billing_project_id=billing_project_id)

Downloading: 100%|██████████| 1756/1756 [00:00<00:00, 2001.84rows/s]


Se eu carregasse o dataframe inteiro sem o WHERE, faria o filtro assim com pandas:

In [98]:
# df['data_inicio'] = pd.to_datetime(df['data_inicio'])
# df_filtrado = df[df['data_inicio'] == '2023-04-01']
# print(df_filtrado)

Mas demoraria muito para carregar, além de poder ser custoso financeiramente.

In [99]:
# Resposta
print(f"Foram abertos {df['id_chamado'].nunique()} chamados no dia 01/04/2023.")

Foram abertos 1756 chamados no dia 01/04/2023.


### 2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?

In [100]:
tipo_chamados = df['tipo'].value_counts()

print(f"O tipo de chamado com mais ocorrências em 01/04/2023 foi '{tipo_chamados.idxmax()}' com {tipo_chamados.max()} chamados.")

O tipo de chamado com mais ocorrências em 01/04/2023 foi 'Estacionamento irregular' com 366 chamados.


### 3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [101]:
df_chamado = df

In [102]:
query_bairro = """SELECT * FROM `datario.dados_mestres.bairro`"""
df_bairro = bd.read_sql(query_bairro, billing_project_id=billing_project_id)

Downloading: 100%|██████████| 164/164 [00:03<00:00, 46.16rows/s]


In [103]:
df_merged = pd.merge(df_chamado, df_bairro, how='left', left_on='id_bairro', right_on='id_bairro')

In [104]:
df_merged.columns

Index(['id_chamado', 'id_origem_ocorrencia', 'data_inicio', 'data_fim',
       'id_bairro', 'id_territorialidade', 'id_logradouro',
       'numero_logradouro', 'id_unidade_organizacional',
       'nome_unidade_organizacional', 'id_unidade_organizacional_mae',
       'unidade_organizacional_ouvidoria', 'categoria', 'id_tipo', 'tipo',
       'id_subtipo', 'subtipo', 'status', 'longitude', 'latitude',
       'data_alvo_finalizacao', 'data_alvo_diagnostico',
       'data_real_diagnostico', 'tempo_prazo', 'prazo_unidade', 'prazo_tipo',
       'dentro_prazo', 'situacao', 'tipo_situacao', 'justificativa_status',
       'reclamacoes', 'data_particao', 'nome', 'id_area_planejamento',
       'id_regiao_planejamento', 'nome_regiao_planejamento',
       'id_regiao_administrativa', 'nome_regiao_administrativa',
       'subprefeitura', 'area', 'perimetro', 'geometry_wkt', 'geometry'],
      dtype='object')

In [105]:
df_merged['nome'].fillna('Bairro Desconhecido', inplace=True)

In [106]:
df_grouped = df_merged.groupby('nome')['id_chamado'].count().reset_index()

In [107]:
df_grouped

,nome,id_chamado
0,Abolição,5
1,Acari,1
2,Alto da Boa Vista,3
3,Anchieta,11
4,Andaraí,10
...,...,...
141,Vila Kosmos,5
142,Vila Militar,3
143,Vila Valqueire,17
144,Vila da Penha,19


In [108]:
df_grouped.rename(columns={'id_chamado': 'chamados'}, inplace=True)

In [109]:
top3_bairros = df_grouped.sort_values(by='chamados', ascending=False).head(3)
print(top3_bairros)

                    nome  chamados
20          Campo Grande       113
128               Tijuca        89
6    Bairro Desconhecido        73


### 4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [110]:
df_grouped = df_merged.groupby('subprefeitura')['id_chamado'].count().reset_index()
df_grouped.rename(columns={'id_chamado': 'chamados'}, inplace=True)
subprefeitura_mais_chamados = df_grouped.loc[df_grouped['chamados'].idxmax()]

In [111]:
print(f"A subprefeitura com mais chamados abertos em 01/04/2023 foi '{subprefeitura_mais_chamados['subprefeitura']}' com {subprefeitura_mais_chamados['chamados']} chamados.")

A subprefeitura com mais chamados abertos em 01/04/2023 foi 'Zona Norte' com 510 chamados.


### 5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

In [112]:
print(top3_bairros[top3_bairros['nome']=='Bairro Desconhecido'])

                  nome  chamados
6  Bairro Desconhecido        73


In [113]:
df_merged[df_merged['id_bairro'].isna()].head()

,id_chamado,id_origem_ocorrencia,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,...,id_area_planejamento,id_regiao_planejamento,nome_regiao_planejamento,id_regiao_administrativa,nome_regiao_administrativa,subprefeitura,area,perimetro,geometry_wkt,geometry
0,18517502,1,2023-04-01 14:34:27,2023-04-01 14:34:27,None,None,None,<NA>,1706,TR/SUBTT/CGLF/CFV - Coordenadoria de Fiscaliza...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18517225,17,2023-04-01 12:44:24,2023-04-01 12:44:24,None,None,None,<NA>,1706,TR/SUBTT/CGLF/CFV - Coordenadoria de Fiscaliza...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18517329,17,2023-04-01 13:24:56,2023-04-01 13:24:56,None,None,None,<NA>,1706,TR/SUBTT/CGLF/CFV - Coordenadoria de Fiscaliza...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18516324,17,2023-04-01 06:56:54,2023-04-01 06:56:54,None,None,None,<NA>,1706,TR/SUBTT/CGLF/CFV - Coordenadoria de Fiscaliza...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18516946,17,2023-04-01 11:13:02,2023-04-01 11:13:02,None,None,None,<NA>,1706,TR/SUBTT/CGLF/CFV - Coordenadoria de Fiscaliza...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Quando id_bairro é vazio, não é possível associar o chamado a nenhum bairro ou subprefeitura.

### 6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [114]:
query_chamado = """
    SELECT *
    FROM `datario.adm_central_atendimento_1746.chamado`
    WHERE DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
    AND subtipo = "Perturbação do sossego"
"""
df_chamado = bd.read_sql(query_chamado, billing_project_id=billing_project_id)

Downloading: 100%|██████████| 42830/42830 [00:12<00:00, 3559.32rows/s]


In [115]:
print(f"Número de chamados com o subtipo 'Perturbação do sossego' de 01/01/2022 até 31/12/2023: {df_chamado['id_chamado'].count()}")

Número de chamados com o subtipo 'Perturbação do sossego' de 01/01/2022 até 31/12/2023: 42830


### 7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).
### e 8. Quantos chamados desse subtipo foram abertos em cada evento?

In [116]:
query_eventos = """ SELECT * FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
"""
df_eventos = bd.read_sql(query_eventos, billing_project_id=billing_project_id)

Downloading: 100%|██████████| 8/8 [00:00<00:00, 31.51rows/s]


In [117]:
df_chamado['data_inicio'] = pd.to_datetime(df_chamado['data_inicio'])
df_chamado['data_fim'] = pd.to_datetime(df_chamado['data_fim'])
df_eventos['data_inicial'] = pd.to_datetime(df_eventos['data_inicial'])
df_eventos['data_final'] = pd.to_datetime(df_eventos['data_final'])

In [118]:
df_chamado['key'] = 1
df_eventos['key'] = 1
merged_df = pd.merge(df_chamado, df_eventos, on='key').drop('key', axis=1)

filtered_df = merged_df[
    (merged_df['data_inicio'] <= merged_df['data_final']) &
    (merged_df['data_fim'] >= merged_df['data_inicial'])
]

filtered_df['evento_ajuste'] = filtered_df['evento'] + ' ' + filtered_df['ano'].astype(str)

unique_df = filtered_df[['id_chamado', 'evento', 'ano', 'evento_ajuste']].drop_duplicates()

In [119]:
eventos_count = unique_df.groupby('evento_ajuste')['id_chamado'].count().reset_index()
eventos_count.columns = ['evento_ajuste', 'chamados']

eventos_count = eventos_count.sort_values(by='chamados', ascending=False)

print(eventos_count)

                            evento_ajuste  chamados
3       Rock in Rio 08/09 a 11/09 de 2022      4289
2       Rock in Rio 02/09 a 04/09 de 2022      4182
1  Reveillon 30-31/12 e 01/01 (2022-2023)       858
0          Carnaval 18/02 a 21/02 de 2023       722


### 9. Qual evento teve a maior média diária de chamados abertos desse subtipo?

In [120]:
filtered_df['duracao_dias'] = (filtered_df['data_final'] - filtered_df['data_inicial']).dt.days + 1

eventos_grouped = filtered_df.groupby(['evento', 'ano']).agg(
    total_chamados=('id_chamado', 'count'),
    duracao_dias=('duracao_dias', 'max')
).reset_index()

eventos_grouped['media_diaria_chamados'] = eventos_grouped['total_chamados'] / eventos_grouped['duracao_dias']

evento_com_maior_media = eventos_grouped.sort_values(by='media_diaria_chamados', ascending=False).iloc[0]

print(f"Evento com a maior média diária de chamados:")
print(f"Evento: {evento_com_maior_media['evento']} {evento_com_maior_media['ano']}")
print(f"Total de Chamados: {evento_com_maior_media['total_chamados']}")
print(f"Duração em Dias: {evento_com_maior_media['duracao_dias']}")
print(f"Média Diária de Chamados: {evento_com_maior_media['media_diaria_chamados']:.2f}")

Evento com a maior média diária de chamados:
Evento: Rock in Rio 02/09 a 04/09 de 2022
Total de Chamados: 8364
Duração em Dias: 3
Média Diária de Chamados: 2788.00


### 10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

In [121]:
dias_totais = (pd.to_datetime('2023-12-31') - pd.to_datetime('2022-01-01')).days + 1
total_chamados_geral = df_chamado[df_chamado['subtipo'] == 'Perturbação do sossego'].shape[0]
media_diaria_geral = total_chamados_geral / dias_totais

eventos_grouped['media_diaria_geral'] = media_diaria_geral

eventos_grouped['diff'] = eventos_grouped['media_diaria_chamados'] - media_diaria_geral
eventos_grouped['percent'] = (eventos_grouped['media_diaria_chamados'] / media_diaria_geral - 1) * 100

eventos_especificos = ['Reveillon', 'Carnaval', 'Rock in Rio']
resultados = eventos_grouped[eventos_grouped['evento'].isin(eventos_especificos)]

resultados = resultados.sort_values(by='media_diaria_chamados', ascending=False)

print(resultados[['evento', 'total_chamados', 'duracao_dias', 'media_diaria_chamados', 'media_diaria_geral', 'diff', 'percent']])

        evento  total_chamados  duracao_dias  media_diaria_chamados  \
2  Rock in Rio            8364             3                 2788.0   
3  Rock in Rio            8578             4                 2144.5   
1    Reveillon            1716             3                  572.0   
0     Carnaval            1444             4                  361.0   

   media_diaria_geral         diff      percent  
2           58.671233  2729.328767  4651.902872  
3           58.671233  2085.828767  3555.113238  
1           58.671233   513.328767   874.924119  
0           58.671233   302.328767   515.293019  


**Fiz o desafio de SQL (analise_sql.sql) duas semanas atrás e aparentemente dados foram adicionados ou modificados nas tabelas, por isso os resultados estão diferentes. Rodei hoje as queries novamente e os resultados batem com esses daqui.**